# Part I: Libraries:

In [106]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
%matplotlib inline 

# Part II: Functions: 

In [44]:
model_df.columns

Index([u'YEAR', u'prodn_practice_desc_IRRIGATED',
       u'prodn_practice_desc_NON-IRRIGATED',
       u'prodn_practice_desc_NON-IRRIGATED, CONTINUOUS CROP',
       u'prodn_practice_desc_NON-IRRIGATED, FOLLOWING SUMMER FALLOW',
       u'prodn_practice_desc_NOT FOLLOWING ANOTHER CROP',
       u'util_practice_desc_GRAIN', u'util_practice_desc_SILAGE',
       u'util_practice_desc_SUGAR', u'YIELD/ACRE',
       ...
       u'COMMODITY_PEANUTS', u'COMMODITY_RICE', u'COMMODITY_RYE',
       u'COMMODITY_SORGHUM', u'COMMODITY_SOYBEANS', u'COMMODITY_SUGARBEETS',
       u'COMMODITY_SUGARCANE', u'COMMODITY_SUNFLOWER', u'COMMODITY_TOBACCO',
       u'COMMODITY_WHEAT'],
      dtype='object', length=836)

In [101]:
#Function 1:
def fit_model_sklearn(df, commodity, model_name):
    """
    INPUT: df (master dataframe); commodity (the type of commodity that one whishes to predict); mode_name (type of model)
    OUT: model (the fitted modle); X_train; X_test; y_train; y_test 
    """
    com_df = df[df['COMMODITY']==commodity]
    y, X = com_df['YIELD/ACRE'], com_df.drop(['COMMODITY', 'YIELD/ACRE'], axis=1)    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
                                     
    if model_name == "Linear Regression":
        model = LinearRegression()
    elif model_name == 'RandomForestRegressor':
        model = RandomForestRegressor(n_estimators=50)
    elif model_name == 'ExtraTreesRegressor':
        model = ExtraTreesRegressor(n_estimators=50)
    elif model_name == 'GradientBoostingRegressor':
        params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 1,'learning_rate': 0.01, 'loss': 'ls'}
        model = GradientBoostingRegressor(**params)
    model.fit(X_train, y_train) 
    return model, X_train, X_test, y_train, y_test

#Function 2: 
def generate_sub_csv(df, commodity):
    """
    INPUT: df (master dataframe); commodities_list (list of commodities)
    """
    path = '/Users/Hsieh/Desktop/persephone/Data/Models/model_yield_{}.csv'
    df[df['commodity_desc']==commodity].to_csv(path.format(commodity))
        
#Function 3: 
def join_dfs(weather_df, commodity):
    path = '/Users/Hsieh/Desktop/persephone/Data/Models/model_yield_{}.csv'
    yield_df = pd.read_csv('/Users/Hsieh/Desktop/persephone/Data/model_yield_{}.csv'.format(commodity))
    
#Function 4: 
def fit_model_sm(df, commodity, model_name):
    """
    INPUT: df (master dataframe); commodity (the type of commodity that one whishes to predict); mode_name (type of model)
    OUT: model (the fitted modle); X_train; X_test; y_train; y_test 
    """
    com_df = df[df["COMMODITY"]==commodity]
    y, X = com_df['YIELD/ACRE'], com_df.drop(['COMMODITY', 'YIELD/ACRE'], axis=1)    
                                     
    if model_name == "Linear Regression":
        X = sm.add_constant(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        model = sm.OLS(y_train, X_train)
        
    results = model.fit()            
    return model, X_train, X_test, y_train, y_test

# Part III: Reorganizing Dataframes: 

In [3]:
#declaring variables: 
targeted_states = pd.Series(["California", "Iowa", "Texas", "Nebraska", "Illinois",\
                  "Minnesota", "Kansas", "Indiana", "North Carolina", "Wisconsin"])
targeted_states = targeted_states.apply(lambda x: x.upper())

## a) yield:

In [29]:
#load yield_csv:
yield_df = pd.read_csv('/Users/Hsieh/Desktop/persephone/Data/cleaned_master_yield.csv')

In [30]:
target_units = ['TONS / ACRE','LB / ACRE']
yield_df = yield_df[yield_df["unit_desc"].apply(lambda x: True if (x in target_units) else (False))]
yield_df = yield_df[yield_df["unit_desc"].notnull()==True]

In [31]:
out_put = ['value']
group_columns = ['year','state_name','county_name','commodity_desc','unit_desc']
dummy_columns = ['prodn_practice_desc','util_practice_desc','class_desc']
drop_columns = ['Unnamed: 0','data_item','state_alpha','statisticcat_desc','asd_code','asd_desc',\
               'congr_district_code','county_ansi','county_code','location_desc']
drop_extra = ['state_name','county_name','year']
#drop unneeded columsn for yield/acre prediction: 
yield_df.sort("year", ascending=True, inplace=True)
yield_df.drop(drop_columns,axis=1,inplace=True)
#yield_df.reset_index(inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [32]:
yield_df['STATE'] = yield_df['state_name']
yield_df['COUNTY'] = yield_df['county_name']
yield_df['YEAR'] = yield_df['year']
yield_df.drop(drop_extra,axis=1,inplace=True)

In [33]:
commodities_list = yield_df.commodity_desc.unique()

In [34]:
#Create  dummy variables:
yield_df= pd.get_dummies(yield_df, columns=['prodn_practice_desc','util_practice_desc'],drop_first=True)
#df_prodn = pd.get_dummies(yield_df['prodn_practice_desc'])
#df_util_practice = pd.get_dummies(yield_df['util_practice_desc'])
#df_class = pd.get_dummies(yield_df['class_desc'])

In [35]:
yield_df.columns

Index([u'commodity_desc', u'unit_desc', u'value', u'class_desc', u'STATE',
       u'COUNTY', u'YEAR', u'prodn_practice_desc_IRRIGATED',
       u'prodn_practice_desc_NON-IRRIGATED',
       u'prodn_practice_desc_NON-IRRIGATED, CONTINUOUS CROP',
       u'prodn_practice_desc_NON-IRRIGATED, FOLLOWING SUMMER FALLOW',
       u'prodn_practice_desc_NOT FOLLOWING ANOTHER CROP',
       u'util_practice_desc_GRAIN', u'util_practice_desc_SILAGE',
       u'util_practice_desc_SUGAR'],
      dtype='object')

In [36]:
#drop variables that have already been dummified: 
yield_df.drop('class_desc',axis=1,inplace=True)

In [37]:
yield_df['unit_desc'].value_counts()

TONS / ACRE    980303
LB / ACRE       95807
Name: unit_desc, dtype: int64

In [38]:
yield_df['COMMODITY'] = yield_df['commodity_desc']
yield_df['UNIT'] = yield_df['unit_desc']
yield_df['YIELD/ACRE'] = yield_df['value']
yield_df.drop(['commodity_desc','unit_desc','value'],axis=1,inplace=True)

In [ ]:
yield_df.to_csv('/Users/Hsieh/Desktop/persephone/Data/model_yield_df_dummies.csv')

In [41]:
commodities_list = yield_df['COMMODITY'].unique()

In [ ]:
#generate_sub_csv(yield_df, commodities_list)

## b) weather: 

In [4]:
#load weather csv:
#weather_df = pd.read_csv('/Users/Hsieh/Desktop/persephone/Data/cleaned_master_weather.csv')
weather_df = pd.read_csv('/Users/Hsieh/Desktop/persephone/Data/cleaned_master_weather_complete.csv')

In [5]:
#declaring variables: 
abnormal = -9999.00
w_group_columns = ['STATE','COUNTY','YEAR','MONTH']

features = ['CLDD','DPNP','DPNT','HTDD','DT90', 'DX32', 'DT00', 'DT32', 'DP01', 'DP05', 'DP10', 'MMXP',\
    'MMNP','TEVP','HO51A0','HO51P0','HO52A0','HO52P0','HO53A0','HO53P0','HO54A0','HO54P0','HO55A0','HO55P0',\
    'HO56A0','HO56P0','HO01A0','HO03A0','LO51A0','LO51P0','LO52A0','LO52P0','LO53A0','LO53P0','LO54A0','LO54P0',\
    'LO55A0','LO55P0','LO56A0','LO56P0','LO01A0','LO03A0','MO51A0','MO51P0','MO52A0','MO52P0','MO53A0','MO53P0',\
    'MO54A0','MO54P0', 'MO55A0','MO55P0','MO56A0','MO56P0','MO01A0','MO03A0','EMXP','MXSD','DSNW','TPCP','TSNW','EMXT',\
    'EMNT','MMXT','MMNT','MNTM','TWND']

#order_columns = ['STATE','COUNTY', 'MONTH','DSNW','EMNT','EMXP','EMXT','MMNT','MMXT','MNTM','MXSD','TPCP','TSNW','YEAR']
model_group_columns = ['STATE','COUNTY','YEAR']

In [6]:
#drop unneeded columns:
weather_df.drop(['Unnamed: 0','LATITUDE','LONGITUDE'],axis=1,inplace=True)
#grab monthly value:
weather_df["MONTH"] = weather_df["DATE"].apply(lambda x: int(str(x)[4:6]))
#turn -9999.00 (the way the data record Nan values) into Nan values:
for feature in features: 
    weather_df[feature] = weather_df[feature].apply(lambda x: np.nan if (x == abnormal) else (x))
#filter out a row where COUNTY value is missing: 
weather_df = weather_df[weather_df["COUNTY"].notnull()==True]
#capitalized 'STATE' and 'COUNTY':
weather_df['STATE'] = weather_df['STATE'].apply(lambda x: x.upper())
weather_df['COUNTY'] = weather_df['COUNTY'].apply(lambda x: x.upper())
#drop unneeded columns: 
weather_df.drop('DATE',axis=1,inplace=True)
#check total null values:
#for c in weather_df:
#    print c, np.mean(pd.isnull(weather_df[c]))
#median weather data in respect to coutny/year:
#median_weather_df = weather_df.groupby(w_group_columns).median().reset_index()
#back fill na for values that is missing in median df:
#median_weather_df.fillna(method="bfill",inplace=True)

In [7]:
#average annual weather day: 
weather_df = weather_df.groupby(['STATE','COUNTY','YEAR']).mean().reset_index()

In [8]:
weather_df.drop(['MONTH'], axis=1, inplace=True)

In [9]:
#drop columns with more 30% na values:
drop_features = []
for c in weather_df:
    if np.mean(pd.isnull(weather_df[c])) > 0.3:
        drop_features.append(c)

In [10]:
weather_df.drop(drop_features, axis=1, inplace=True)

In [11]:
print drop_features

['MMXP', 'MMNP', 'TEVP', 'HO51A0', 'HO51P0', 'HO52A0', 'HO52P0', 'HO53A0', 'HO53P0', 'HO54A0', 'HO54P0', 'HO55A0', 'HO55P0', 'HO56A0', 'HO56P0', 'HO01A0', 'HO03A0', 'LO51A0', 'LO51P0', 'LO52A0', 'LO52P0', 'LO53A0', 'LO53P0', 'LO54A0', 'LO54P0', 'LO55A0', 'LO55P0', 'LO56A0', 'LO56P0', 'LO01A0', 'LO03A0', 'MO51A0', 'MO51P0', 'MO52A0', 'MO52P0', 'MO53A0', 'MO53P0', 'MO54A0', 'MO54P0', 'MO55A0', 'MO55P0', 'MO56A0', 'MO56P0', 'MO01A0', 'MO03A0', 'TWND']


In [12]:
#since the df is order based on time and location this will fill the na with the values of next year. 
weather_df.fillna(method="bfill", inplace=True)

In [13]:
weather_df.to_csv('/Users/Hsieh/Desktop/persephone/Data/model_weather_df_complete.csv')

# Part IV: Combining Dataframes:

In [54]:
#weather_model_df = pd.read_csv('/Users/Hsieh/Desktop/persephone/Data/model_weather_df.csv')
weather_model_df = pd.read_csv('/Users/Hsieh/Desktop/persephone/Data/model_weather_df_complete.csv')
yield_model_df = pd.read_csv('/Users/Hsieh/Desktop/persephone/Data/model_yield_df_dummies.csv')

In [55]:
model_df = pd.merge(left=yield_model_df, right=weather_model_df, left_on=['STATE','COUNTY','YEAR'],\
                   right_on=['STATE','COUNTY','YEAR'], how="inner")

In [56]:
model_df.columns

Index([u'Unnamed: 0_x', u'STATE', u'COUNTY', u'YEAR',
       u'prodn_practice_desc_IRRIGATED', u'prodn_practice_desc_NON-IRRIGATED',
       u'prodn_practice_desc_NON-IRRIGATED, CONTINUOUS CROP',
       u'prodn_practice_desc_NON-IRRIGATED, FOLLOWING SUMMER FALLOW',
       u'prodn_practice_desc_NOT FOLLOWING ANOTHER CROP',
       u'util_practice_desc_GRAIN', u'util_practice_desc_SILAGE',
       u'util_practice_desc_SUGAR', u'COMMODITY', u'UNIT', u'YIELD/ACRE',
       u'Unnamed: 0_y', u'CLDD', u'DPNP', u'DPNT', u'HTDD', u'DT90', u'DX32',
       u'DT00', u'DT32', u'DP01', u'DP05', u'DP10', u'EMXP', u'MXSD', u'DSNW',
       u'TPCP', u'TSNW', u'EMXT', u'EMNT', u'MMXT', u'MMNT', u'MNTM'],
      dtype='object')

In [57]:
model_df.drop(['Unnamed: 0_x','Unnamed: 0_y','UNIT'], axis=1, inplace=True)

In [59]:
#grouping column names: 
categorical_features = ['STATE','COUNTY',]
dependent_variable = ['YIELD/ACRE']
#get dummy variables for categorical variables:
model_df= pd.get_dummies(model_df, columns=categorical_features,drop_first=True)
model_df.to_csv('/Users/Hsieh/Desktop/persephone/Data/model_df_complete.csv')

# Part V: Features Engineering: 

In [51]:
#N/A

# Part VI: Models: 

In [89]:
#after cleanning, some commodities has no values (removing these commodities)
commodities_list = list(commodities_list)
commodities_list.remove('SAFFLOWER')
commodities_list.remove('MUSTARD')
commodities_list.remove('LENTILS')
commodities_list.remove('PEAS')

## a: RandomForestRegressor: 

In [ ]:
#RandomForestRegression:
#model, X_train, X_test, y_train, y_test = fit_model_sklearn(model_df, "MUSTARD", "RandomForestRegressor")
#predict = model.predict(X_test)
#print r2_score(y_test, predict)

In [91]:
#results for all commodities:
for commodity in commodities_list: 
    model, X_train, X_test, y_train, y_test = fit_model_sklearn(model_df, commodity, "RandomForestRegressor")
    predict = model.predict(X_test)
    print "****************************************************************************"
    print "{}'s adjusted r^2 score with {} is:".format(commodity,"RandomForestRegressor")
    print r2_score(y_test, predict)

****************************************************************************
WHEAT's adjusted r^2 score with RandomForestRegressoris:
0.911247406595
****************************************************************************
HAY's adjusted r^2 score with RandomForestRegressoris:
0.29154540912
****************************************************************************
CORN's adjusted r^2 score with RandomForestRegressoris:
0.878642694583
****************************************************************************
OATS's adjusted r^2 score with RandomForestRegressoris:
0.629691443376
****************************************************************************
BARLEY's adjusted r^2 score with RandomForestRegressoris:
0.592831648257
****************************************************************************
TOBACCO's adjusted r^2 score with RandomForestRegressoris:
0.300607453061
****************************************************************************
SOYBEANS's adjusted r^2 score w

## b: Linear Regression: 

### i) w/ Sklearn: 

In [ ]:
#fitting: 
#model, X_train, X_test, y_train, y_test = fit_model_sklearn(model_df, "BARLEY", "Linear Regression")
#predict = model.predict(X_test)
#print r2_score(y_test, predict)
#scoring: 
#print r2_score(y_test, predict)

### ii) w/ Stats Models: 

In [ ]:
#fitting: 
#model, X_train, X_test, y_train, y_test = fit_model_sm(model_df, "BARLEY", "Linear Regression"):
#y_predict = results.predict(X_test)

In [ ]:
#scoring: 
#print results.summary()

In [110]:
#results for all commodities:
#for commodity in commodities_list: 
model, X_train, X_test, y_train, y_test = fit_model_sm(model_df, commodity, "Linear Regression")
predict = model.predict(X_test)
#    print "****************************************************************************"
#    print "{}'s adjusted r^2 score with {} is:".format(commodity,"Linear Regression")
#    print results.summary()
#    print r2_score(y_test, predict)

ValueError: shapes (85692,818) and (42207,818) not aligned: 818 (dim 1) != 42207 (dim 0)

## c: ExtraTreesRegressor:

In [111]:
#results for all commodities:
for commodity in commodities_list: 
    model, X_train, X_test, y_train, y_test = fit_model_sklearn(model_df, commodity, "ExtraTreesRegressor")
    predict = model.predict(X_test)
    print "****************************************************************************"
    print "{}'s adjusted r^2 score with {} is:".format(commodity,"ExtraTreesRegressor")
    print r2_score(y_test, predict)

****************************************************************************
WHEAT's adjusted r^2 score with ExtraTreesRegressor is:
0.924089785802
****************************************************************************
HAY's adjusted r^2 score with ExtraTreesRegressor is:
0.151869505068
****************************************************************************
CORN's adjusted r^2 score with ExtraTreesRegressor is:
0.899585625326
****************************************************************************
OATS's adjusted r^2 score with ExtraTreesRegressor is:
0.67552886568
****************************************************************************
BARLEY's adjusted r^2 score with ExtraTreesRegressor is:
0.631318013723
****************************************************************************
TOBACCO's adjusted r^2 score with ExtraTreesRegressor is:
0.386825483183
****************************************************************************
SOYBEANS's adjusted r^2 score with Ex

## d: Gradient Boosting: 

In [103]:
#results for all commodities:
#for commodity in commodities_list: 
#    model, X_train, X_test, y_train, y_test = fit_model_sklearn(model_df, commodity, "GradientBoostingRegressor")
#    predict = model.predict(X_test)
#    print "****************************************************************************"
#    print "{}'s adjusted r^2 score with {} is:".format(commodity,"GradientBoostingRegressor")
#    print r2_score(y_test, predict)

# Part VI: Grid Search: 

## a): Linear Regression: 

## b) Random Forest: 

## c) Gradient Boosting: 

# Part VII: Score: 

# Part VIII: Conclusion: 

# Archive Code:

In [ ]:
#Function 1: 
"""
def merge_cols(x):
    return x['COUNTY'] + x['STATE'] + str(x['MONTH'])
"""

#Function 2: 
"""
def fill_na(x, median_df):
    INPUT: x (pd series; row of a df); median_df (pd df; dataframe with the needed average values)
    OUTPUT: new_x (pd series; new row of df with filled_na values)
    OVERVIEW: fill in na values of a df with historical average of the monthly value of that month and region 
    
    state, county, month = x[0], x[1], x[2]  
    
    row = median_df.loc[(median_df['STATE']==state)&(median_df['COUNTY']==county)&(median_df['MONTH']==month)]
    s_row = row.iloc[0][1:]  
    
    x.loc[np.where(x=="NAN")] = s_row.loc[np.where(x=="NAN")]
    return x 
"""
#model_df.unit_desc.unique()

#grouping column names: 
#categorical_features = ['STATE','COUNTY']
#numeric_features = ['YEAR','DSNW','EMNT','EMXP','EMXT','MMNT','MMXT','MNTM','MXSD','TPCP','TSNW']
#dependent_variable = ['value']

# Features Key: 

In [26]:
features_key = {'CLDD':'Cooling degree days','DPNP':'Departure from normal monthly precipitation',\
               'DPNT':'Departure from normal monthly temperature','HTDD':'Heating degree days',\
                'DT90':'Number days with maximum temperature greater than or equal 90.0 F',\
                'DX32':'Number days with maximum temperature less than or equal to 32.0 F',\
                'DT00':'Number days with minimum temperature less than or equal to 0.0 F',\
                'DT32':'Number days with minimum temperature less than or equal to 32.0 F',\
                'DP01':'Number of days with greater than or equal to 0.1 inch of precipitation',\
                'DP05':'Number of days with greater than or equal to 0.5 inch of precipitation',\
                'DP10':'Number of days with greater than or equal to 1.0 inch of precipitation',\
                'MMXP':'Monthly mean maximum temperature of evaporation pan water',\
                'MMNP':'Monthly mean maximum temperature of evaporation pan water',\
                'TEVP':'Total monthly evaporation',\
                'EMXP':'Extreme maximum daily precipitation','MXSD':'Maximum snow depth',\
                'DSNW':'Number days with snow depth > 1 inch','TPCP':'Total precipitation',\
                'TPCP':'Total precipitation','TSNW':'Total snow fall',\
                'EMXT':'Extreme maximum daily temperature','EMNT':'Extreme maximum daily temperature',\
                'EMNT':'Extreme minimum daily temperature','MMXT':'Monthly Mean maximum temperature',\
                'MMNT':'Monthly Mean minimum temperature','MNTM':' Monthly mean temperature',\
                'TWND':'Total monthly wind movement over evaporation pan'}